First, we use Aswath Damodaran's (NYU Stern prof) open data set.

Yes, we are allowed to use this. See his [philosophy](https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datahistory.html)
here.

Instead of working with Excel, let's read from the
[HTML chart on his
website](https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html).
It's a bit funky but it works.

In [ ]:
import urllib.request, ssl

DATA_SOURCE_URL = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html"

# https returns errors for some reason
# time to be unsafe...
with urllib.request.urlopen(DATA_SOURCE_URL,
                            context=ssl._create_unverified_context()) as response:
    html_content = response.read().decode()

Good. Check over the data to make sure it's sane. Then continue.

In [3]:
print(html_content)

<html xmlns:o="urn:schemas-microsoft-com:office:office"
xmlns:x="urn:schemas-microsoft-com:office:excel"
xmlns="http://www.w3.org/TR/REC-html40">

<head>
<meta http-equiv=Content-Type content="text/html; charset=macintosh">
<meta name=ProgId content=Excel.Sheet>
<meta name=Generator content="Microsoft Excel 15">
<link rel=File-List href="histimpl.fld/filelist.xml">
<style>
<!--table
	{mso-displayed-decimal-separator:"\.";
	mso-displayed-thousand-separator:"\,";}
@page
	{margin:.75in .7in .75in .7in;
	mso-header-margin:.3in;
	mso-footer-margin:.3in;}
.style0
	{mso-number-format:General;
	text-align:general;
	vertical-align:bottom;
	white-space:nowrap;
	mso-rotate:0;
	mso-background-source:auto;
	mso-pattern:auto;
	color:black;
	font-size:12.0pt;
	font-weight:400;
	font-style:normal;
	text-decoration:none;
	font-family:Calibri, sans-serif;
	mso-font-charset:0;
	border:none;
	mso-protection:locked visible;
	mso-style-name:Normal;
	mso-style-id:0;}
.style20
	{mso-number-format:0%;
	mso-sty

To avoid needing to `pip install` other libraries, we
HTML parse by hand. It's predictable output anyway (from an Excel to HTML converter -- check the comments of the HTML source)

In [4]:
import html.parser

class MarketData:
    def add(self, index, value):
        setattr(self, self.index_to_property(index), value)
    def index_to_property(cls, index):
        return {
            0: "year",
            1: "snp500_growth_with_dividends",
            3: "t10bond",
            6: "gold",
            16: "inflation",
        }[index]
    def __str__(self):
        return f"[{self.year},{self.snp500_growth_with_dividends},{self.t10bond},{self.gold},{self.inflation}]"

class Parser(html.parser.HTMLParser):
    def __init__(self):
        super().__init__()
        # 0: Year
        # 1: S&P500 (includes dividends) | ETFs pay dividends anyway
        # 3: US T. Bond (10-year)        | Treasury bond
        # 6: Gold                        | *Gold standard made this
        # 16: Inflation rate               *      stable after 1971
        self.collated_data = []
        self.inside_cell = False
        self.current_row = 0 # ignore first 2 rows
        self.current_index = 0
        self.desired_indices = [0, 1, 3, 6, 16]

    def handle_starttag(self, tag, attrs):
        if tag == "tr" and ("style", "height:16.0pt") in attrs:
            self.current_row += 1
            if self.current_row >= 2:
               self.collated_data.append(MarketData())
            else:
                print("skipped row")
            self.current_index = 0
            self.inside_cell = False
        elif tag == "td" and self.current_row >= 2:
            self.inside_cell = True

    def handle_data(self, data):
        data = data.strip()
        if self.inside_cell and data not in ["\n", "$", ""]:
            print(f"{self.current_index}: {data} (length {len(data)})")
            if self.current_row >= 2 and self.current_index in self.desired_indices:
                if "%" in data:
                    data = data[:-1]
                # numerize
                if self.current_index == 0:
                    data = int(data)
                else:
                    data = float(data)
                self.collated_data[-1].add(self.current_index, data)
            self.current_index += 1
        


parser = Parser()
parser.feed(html_content)
print([item.__dict__ for item in parser.collated_data])

skipped row
0: 1928 (length 4)
1: 43.81% (length 6)
2: 3.08% (length 5)
3: 0.84% (length 5)
4: 3.22% (length 5)
5: 1.49% (length 5)
6: 0.10% (length 5)
7: 143.81 (length 6)
8: 103.08 (length 6)
9: 100.84 (length 6)
10: 103.22 (length 6)
11: 101.49 (length 6)
12: 100.10 (length 6)
13: 40.73% (length 6)
14: 42.98% (length 6)
15: 40.59% (length 6)
16: -1.16% (length 6)
17: 45.49% (length 6)
18: 4.29% (length 5)
19: 2.01% (length 5)
20: 4.43% (length 5)
21: 2.68% (length 5)
22: 1.27% (length 5)
0: 1929 (length 4)
1: -8.30% (length 6)
2: 3.16% (length 5)
3: 4.20% (length 5)
4: 3.02% (length 5)
5: -2.06% (length 6)
6: -0.15% (length 6)
7: 131.88 (length 6)
8: 106.34 (length 6)
9: 105.07 (length 6)
10: 106.33 (length 6)
11: 99.40 (length 5)
12: 99.95 (length 5)
13: -11.46% (length 7)
14: -12.50% (length 7)
15: -11.32% (length 7)
16: 0.58% (length 5)
17: -8.83% (length 6)
18: 2.56% (length 5)
19: 3.60% (length 5)
20: 2.42% (length 5)
21: -2.63% (length 6)
22: -0.73% (length 6)
0: 1930 (length 

After sanity checking, write it out to a JSON.

In [ ]:
import json
stuff = [o.__dict__ for o in parser.collated_data]
assert(len(stuff) > 5) # sanity check on size

with open("MatchTheMarket.json", "w") as f:
    json.dump(stuff, f, indent=2)

with open("MatchTheMarket.json", "r") as f:
    print(f.read())

[
  {
    "year": 1928,
    "snp500_growth_with_dividends": 43.81,
    "t10bond": 0.84,
    "gold": 0.1,
    "inflation": -1.16
  },
  {
    "year": 1929,
    "snp500_growth_with_dividends": -8.3,
    "t10bond": 4.2,
    "gold": -0.15,
    "inflation": 0.58
  },
  {
    "year": 1930,
    "snp500_growth_with_dividends": -25.12,
    "t10bond": 4.54,
    "gold": 0.1,
    "inflation": -6.4
  },
  {
    "year": 1931,
    "snp500_growth_with_dividends": -43.84,
    "t10bond": -2.56,
    "gold": -17.38,
    "inflation": -9.32
  },
  {
    "year": 1932,
    "snp500_growth_with_dividends": -8.64,
    "t10bond": 8.79,
    "gold": 21.28,
    "inflation": -10.27
  },
  {
    "year": 1933,
    "snp500_growth_with_dividends": 49.98,
    "t10bond": 1.86,
    "gold": 27.26,
    "inflation": 0.76
  },
  {
    "year": 1934,
    "snp500_growth_with_dividends": -1.19,
    "t10bond": 7.96,
    "gold": 31.75,
    "inflation": 1.52
  },
  {
    "year": 1935,
    "snp500_growth_with_dividends": 46.74,
    "t1